In [1]:
cd /home/ubuntu/DistMEC/

/home/ubuntu/DistMEC


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

from classes.User import *
from classes.solver import *
from classes.Server import *
from classes.distributed_utils import *

from tqdm import tqdm, tqdm_notebook
from time import sleep
import pickle
import os
import numpy as np

import time


In [3]:
# Recording Mechanism
    
# trial_list = [3,5,10,15,20]
trial_list = [40]
dist_list = []
cent_list = []
    
for tri in range(len(trial_list)): 
    
    # System Parameters
    T = 3000
    num_users = trial_list[tri]
    num_svrs = trial_list[tri]
    num_locs = 4
    mu = np.random.uniform(low=0, high = 1, size = [num_users, num_svrs])
    ceiling = 40
    # w = np.random.uniform(low=0, high=1, size= [num_users, num_svrs])
    space_1d_dim = 10 #4 for small, 10 for big
    svr_offset_uniform = 2
    max_dist = 8

    num_explore = 1

    # result dictionaries
    result_dicts = []
    performance_list = []
    user_dicts = []

    # Generate visitng locations and server locations
    usr_locs = gen_eq_locs(space_1d=space_1d_dim, nums=num_locs,offset = 1.7)

    svr_locs = gen_rand_locs(space_1d=space_1d_dim - svr_offset_uniform, nums=num_svrs)
    # Create Users
    Users_w = []
    for i in range(num_users):
        Users_w += [User(usr_locs,svr_locs,mu[i],i,
                      max_dist = max_dist, threshold_dist = 6, self_weight = 1 - 0.95, P = None, ceiling = 1,
                      sticky_mode = True, kick_mode=True)]
    
    # Create Servers
    Servers = []
    for i in range(num_svrs):
        Servers += [Server(svr_locs[i],mu,i)]
    
    # Recorded Values - reservation
    regret_g = np.zeros(T) # Got
    collision_count_g = np.zeros(T)
    optimal_reward_g = np.zeros(T)

    regret_w = np.zeros(T) # Worst
    collision_count_w = np.zeros(T)
    optimal_reward = np.zeros(T)
    
    # Explore rounds are common/shared across all users
    explore_rounds(Users_w, num_users, Servers, mu, regret_w, collision_count_w, optimal_reward,
                   usr_move_flag = True, rounds=num_explore)

    round_start = ((num_svrs)*num_explore)+1

    
    # Other reward recodings
    Users_rsv = copy.deepcopy(Users_w)
    regret_rsv = copy.deepcopy(regret_w)
    collision_count_rsv = copy.deepcopy(collision_count_w)
    for u in Users_rsv:
        u.sticky_mode = True
        u.kick_mode = True #false
        u.ceiling = ceiling
        u.expected_time_true = u.get_expected_time()

    w = obtain_w(Users_w, len(Users_w), len(Servers))
    optimal = offline_optimal_action(w, mu)
    start_time = time.time()

    for (zzz,t) in zip(tqdm_notebook(range(T-round_start)),range(round_start, T)):
        w = obtain_w(Users_w, len(Users_w), len(Servers))
#         optimal = offline_optimal_action(w, mu)
#         optimal_reward[t] = optimal[1]
        
        # Distributed solution 

        play_round(Users_rsv, Servers, mu, regret_rsv, collision_count_rsv, 
                   usr_move_flag = True, debugger = False, reservation_mode = True, optimal =optimal, t = t)

    del Users_rsv, Users_w
    dist_list += [start_time - time.time()]

<ipython-input-3-bb3cd60a9c9c>:75: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for (zzz,t) in zip(tqdm_notebook(range(T-round_start)),range(round_start, T)):


In [4]:
# Recording Mechanism
    
# trial_list = [3,5,10,15,20]
trial_list = [40]
    
for tri in range(len(trial_list)): 
    
    # System Parameters
    T = 3000
    num_users = trial_list[tri]
    num_svrs = trial_list[tri]
    num_locs = 4
    mu = np.random.uniform(low=0, high = 1, size = [num_users, num_svrs])
    ceiling = 40
    # w = np.random.uniform(low=0, high=1, size= [num_users, num_svrs])
    space_1d_dim = 10 #4 for small, 10 for big
    svr_offset_uniform = 2
    max_dist = 8

    num_explore = 1

    # result dictionaries
    result_dicts = []
    performance_list = []
    user_dicts = []

    # Generate visitng locations and server locations
    usr_locs = gen_eq_locs(space_1d=space_1d_dim, nums=num_locs,offset = 1.7)

    with open('traces_P_40k.pkl', 'rb') as handle:
        P_collection = pickle.load(handle)

    with open('traces_P_40k_dict.pkl', 'rb') as handle:
        P_dict = pickle.load(handle)
    
    svr_locs = gen_rand_locs(space_1d=space_1d_dim - svr_offset_uniform, nums=num_svrs)
    # Create Users
    Users_w = []
    for i in range(num_users):
        Users_w += [User(usr_locs,svr_locs,mu[i],i,
                      max_dist = max_dist, threshold_dist = 6, self_weight = 1 - 0.95, P = None, ceiling = 1,
                      sticky_mode = True, kick_mode=True)]

    
    # Create Servers
    Servers = []
    for i in range(num_svrs):
        Servers += [Server(svr_locs[i],mu,i)]
    
    # Recorded Values - reservation
    regret_g = np.zeros(T) # Got
    collision_count_g = np.zeros(T)
    optimal_reward_g = np.zeros(T)

    regret_w = np.zeros(T) # Worst
    collision_count_w = np.zeros(T)
    optimal_reward = np.zeros(T)
    
    # Explore rounds are common/shared across all users
    explore_rounds(Users_w, num_users, Servers, mu, regret_w, collision_count_w, optimal_reward,
                   usr_move_flag = True, rounds=num_explore)

    round_start = ((num_svrs)*num_explore)+1


    # Other reward recodings
    Users_rsv = copy.deepcopy(Users_w)
    regret_rsv = copy.deepcopy(regret_w)
    collision_count_rsv = copy.deepcopy(collision_count_w)
    for u in Users_rsv:
        u.sticky_mode = True
        u.kick_mode = True #false
        u.ceiling = ceiling
        u.expected_time_true = u.get_expected_time()

    # Centralized - true w learning
    rewards_record_ct, pulls_record_ct, ucb_ct = extract_centralized_case(Users_w, num_users, num_svrs)
    regret_ct = copy.deepcopy(regret_w)
    
    # Centralized - stationary w learning
    rewards_record_cs, pulls_record_cs, ucb_cs = extract_centralized_case(Users_w, num_users, num_svrs)
    regret_cs = copy.deepcopy(regret_w)

    # Centralized - stationary w, known mu
    w_stat = obtain_w_stationary(Users_w, num_users, num_svrs)
    optimal_stat_arms = offline_optimal_action(w_stat,mu)[0]
    regret_cst = copy.deepcopy(regret_w)

    start_time = time.time()

    for (zzz,t) in zip(tqdm_notebook(range(T-round_start)),range(round_start, T)):
        w = obtain_w(Users_w, len(Users_w), len(Servers))
#         Centralized Solution -  true w learning
        ucb_ct = update_ucb(rewards_record_ct, pulls_record_ct, ucb_ct, t, 1)
        arms = offline_optimal_action(w, ucb_ct)[0]
        rewards_record_ct, pulls_record_ct = pull_super_arm(arms, mu, rewards_record_ct, pulls_record_ct)
        regret_ct[t] = optimal[1] - expected_reward(arms, mu, w)

    del Users_rsv, Users_w
    cent_list += [start_time - time.time()]

<ipython-input-4-117c7323d4a1>:91: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for (zzz,t) in zip(tqdm_notebook(range(T-round_start)),range(round_start, T)):


In [13]:
8*60 + 40

520

In [6]:
dist_list

[-25.567548990249634]

In [5]:
cent_list

[-523.5170087814331]